<a href="https://colab.research.google.com/github/Cutie-tee/lab-babyagi-with-agent/blob/main/lab_babyagi_with_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [ ]:
!pip install google-generativeai
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
%env GOOGLE_API_KEY=your-api-key

env: GOOGLE_API_KEY=your-api-key


In [ ]:
!pip install --upgrade langchain-google-genai # Upgrade langchain-google-genai to the latest version

from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
#from langchain_openai import OpenAI, OpenAIEmbeddings # Remove OpenAI import

# Import Google Gemini LLM
from langchain_google_genai import GoogleGenerativeAI
# Instead of importing GooglePalmEmbeddings, try using GooglePalmEmbeddings directly:
from langchain.embeddings import GooglePalmEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [ ]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
from google.colab import userdata

In [ ]:
import os

GOOGLE_API_KEY  = userdata.get('GOOGLE_API_KEY')
SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

# Check if SERPAPI_API_KEY is None before setting it in os.environ
if SERPAPI_API_KEY is not None:
    os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY
else:
    # Handle the case where SERPAPI_API_KEY is None, for example:
    print("Warning: SERPAPI_API_KEY is not set. Some features might not work.")
    # You could also raise an exception or set a default value here

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

GOOGLE_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [ ]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.3 MB/s eta 0:00:00


In [ ]:
# Define your embedding model
embeddings_model = GooglePalmEmbeddings(google_api_key=os.environ["GOOGLE_API_KEY"])
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=e55f4957f96bd3bad053f7f1fce78788bd234b576923e639dcde85185f460273
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent #this
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper
from langchain_google_genai import GoogleGenerativeAI  # for the Google model

# Import and rename GoogleGenerativeAI to google_genai
from langchain_google_genai import GoogleGenerativeAI as google_genai

# Get the Google API key from the environment
import os
google_api_key = os.environ["GOOGLE_API_KEY"]

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
#todo_chain = LLMChain(llm=google_genai(temperature=0), prompt=todo_prompt) -i commented this out due to error i received
# Pass the google_api_key when initializing google_genai
todo_chain = LLMChain(llm = google_genai(model="gemini-pro", google_api_key=google_api_key), prompt=todo_prompt) #added this gooten from the langchain documentation
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

<ipython-input-13-271fc80963da>:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm = google_genai(model="gemini-pro", google_api_key=google_api_key), prompt=todo_prompt) #added this gooten from the langchain documentation


In [ ]:
#llm = GoogleGenerativeAI(temperature=0)
llm = google_genai(model="gemini-pro", temperature=0, google_api_key=google_api_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

<ipython-input-14-84759d62fe8a>:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [ ]:
OBJECTIVE = "Write a weather report for SF today"

In [ ]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [ ]:
!pip install langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.4
    Uninstalling pydantic-2.10.4:
      Successfully uninstalled pydantic-2.10.4
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: google

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [ ]:
!pip install google-generativeai --upgrade

In [ ]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


AttributeError: module 'google.generativeai' has no attribute 'generate_embeddings'